In [36]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import json 
import itertools
from datetime import date

## Explore data and structure of the website (Header structure)

In [37]:
url = 'https://www.realestate.com.kh/buy'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

category_url = []

response = requests.get(url, headers=headers)
data = bs(response.content, 'html.parser')


## Get locations list from the website

In [40]:
realestate_location = []
get_lists_of_locations = data.find_all(class_='e1ope10j0 css-ib3p51')
get_lists_of_locations

for location in get_lists_of_locations:
    realestate_location.append(location.get_text())
realestate_location

[]

## Explore on pages listing item

In [19]:
# base_url: https://www.realestate.com.kh/buy/?page=2
# items-container
get_container_element = data.find_all(class_='item css-16n7mg5 eq4or9x0')

In [20]:
# address
addresses = []
for container_info in get_container_element:
    addresses.append(container_info.find_all(class_='address'))
addresses

[[<div class="address"><span class="location-pin"></span>Veal Sbov, Chbar Ampov, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span>Tonle Bassac, Chamkarmon, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span>Koh Dach, Chroy Changvar, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span>Ream, Prey Nob, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span>Chey Chumneah, Daun Penh, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span>Preaek Lieb, Chroy Changvar, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Buon, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Bei, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Bei, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> <

In [21]:
# get name
names = []
for container_info in get_container_element:
    names.append(container_info.find_all(class_='featured css-1omkpog e1jqslr40'))

In [22]:
# get price value
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    pprint(container_info.find_all(class_='price'))


 ========================================= ========================================= 

[<div class="price"><span class="value" style="text-decoration:">$236,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$840,403</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$150/m²</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$1,150,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$52,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$275,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price

In [23]:
# get info bedroom, bathroom
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    pprint(container_info.find_all(class_='private'))


 ========================================= ========================================= 

[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 8</span><span class="short-label">8</span></div><div class="feature"><span class="icon"></span> <span class="label">Bathrooms: 10</span><span class="short-label">10</span></div></div>]
[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 3</span><span class="short-label">3</span></div><div class="feature"><span class="icon"></span> <span class="label">Bathrooms: 3</span><span class="short-label">3</span></div></div>]
[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Land area: 792m²</span><span class="short-label">792m²</span></div></div>]
[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 3</span><span class="short-label">3</span></div><div class="feature"><spa

In [24]:
# get type
type_list = []
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    # pprint(container_info.find(class_='heading'))
    type_list.append(container_info.find(class_='heading'))
    if container_info.find("div", {"class":"heading"}):
        print(container_info.find("div", {"class":"heading"}).get_text().replace("$"," ").replace(" "," ").split(" ")[0])
    
# type(type_list)


 ========================================= ========================================= 

Shophouse
Condo
Land
Penthouse
Apartment
Twin
Apartment
Land
Land
Project
Project
Apartment
Condo
Condo
Project
Borey
Condo
Project


## Function to extract the information

In [53]:
test = []
for x in range(len(addresses)):
    # Regexr area to filter data    
    bedroom_regexr = re.compile('Bedrooms: (\d+)')
    bedrooms_regexr = re.compile('Bedrooms: (\d+) to (\d+)')
    
    bathroom_regexr = re.compile('Bathrooms: (\d+)')
    bathrooms_regexr = re.compile('Bathrooms: (\d+) to (\d+)')

    floor_regexr = re.compile('Floor area: (\d+)')
    floors_regexr = re.compile('Floor area: (\d+) to (\d+)')

    
    land_regexr = re.compile('Land area: (\d+)')
    lands_regexr = re.compile('Land area: (\d+) t0 (\d+)')
    #============== End here ============================
    
    # Variable area
    realestate_type = get_container_element[x].find("div", {"class":"heading"})
    address = get_container_element[x].find(class_='address').get_text().replace('\ue92f','')
    
    price = get_container_element[x].find(class_='price').get_text().replace('\ue92f','').replace(' ','')
    
    feature = get_container_element[x].find_all(class_='feature')
    
    bedroom = bedroom_regexr.findall(str(feature))[0] if bedroom_regexr.findall(str(feature)) else None
    
    bedrooms = bedrooms_regexr.findall(str(feature))[0] if bedrooms_regexr.findall(str(feature)) else None

    
    bathroom = bathroom_regexr.findall(str(feature))[0] if bathroom_regexr.findall(str(feature)) else None
    
    bathrooms = bathrooms_regexr.findall(str(feature))[0] if bathrooms_regexr.findall(str(feature)) else None

    link = get_container_element[x].find(class_='shadow').find("a")['href']
        
    floor = floor_regexr.findall(str(feature))[0] if floor_regexr.findall(str(feature)) else None
    floors = floors_regexr.findall(str(feature))[0] if floors_regexr.findall(str(feature)) else None

    land = land_regexr.findall(str(feature))[0] if land_regexr.findall(str(feature)) else None
    lands = lands_regexr.findall(str(feature))[0] if lands_regexr.findall(str(feature)) else None
    
    realestate_type = realestate_type.get_text().replace("$"," ").replace(" "," ").split(" ")[0] if realestate_type else None
    #============== End here ============================

    # conditional before push in list
    final_bedroom = bedrooms if bedrooms else bedroom
    final_bathroom = bathrooms if bathrooms else bathroom
    final_floor = floors if floors else floor
    final_land = lands if lands else land
    #============== End here ============================
    test.append({'location': address, 'price': price, 'bedrooms': final_bedroom,
                 'bathrooms': final_bathroom, 'floors': final_floor, 'lands': final_land,
                 'type': realestate_type, 'link': 'https://www.realestate.com.kh'+link
                })
test



[{'location': 'Veal Sbov, Chbar Ampov, Phnom Penh',
  'price': '$236,000',
  'bedrooms': '8',
  'bathrooms': '10',
  'floors': None,
  'lands': None,
  'type': 'Shophouse',
  'link': 'https://www.realestate.com.kh/buy/veal-sbov/8-bed-10-bath-shophouse-232724/'},
 {'location': 'Tonle Bassac, Chamkarmon, Phnom Penh',
  'price': '$840,403',
  'bedrooms': '3',
  'bathrooms': '3',
  'floors': None,
  'lands': None,
  'type': 'Condo',
  'link': 'https://www.realestate.com.kh/buy/tonle-bassac/3-bed-3-bath-condo-232673/'},
 {'location': 'Koh Dach, Chroy Changvar, Phnom Penh',
  'price': '$150/m²',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '792',
  'type': 'Land',
  'link': 'https://www.realestate.com.kh/buy/koh-dach/land-232168/'},
 {'location': 'Ream, Prey Nob, Sihanoukville',
  'price': '$1,150,000',
  'bedrooms': '3',
  'bathrooms': '5',
  'floors': '704',
  'lands': None,
  'type': 'Penthouse',
  'link': 'https://www.realestate.com.kh/buy/ream/3-bed-5-bath-penth

## Convert the loop to function

In [54]:
realestate_data_collection  = []

def extra_name_location_price_info(url, headers, sell_or_rent):
    response = requests.get(url, headers=headers)
    data = bs(response.content, 'html.parser')
    get_container_element = data.find_all(class_='item css-16n7mg5 eq4or9x0')
    for x in range(len(addresses)):
        # Regexr area to filter data    
        bedroom_regexr = re.compile('Bedrooms: (\d+)')
        bedrooms_regexr = re.compile('Bedrooms: (\d+) to (\d+)')

        bathroom_regexr = re.compile('Bathrooms: (\d+)')
        bathrooms_regexr = re.compile('Bathrooms: (\d+) to (\d+)')

        floor_regexr = re.compile('Floor area: (\d+)')
        floors_regexr = re.compile('Floor area: (\d+) to (\d+)')


        land_regexr = re.compile('Land area: (\d+)')
        lands_regexr = re.compile('Land area: (\d+) t0 (\d+)')
        #============== End here ============================

        # Variable area
        realestate_type = get_container_element[x].find("div", {"class":"heading"})
        address = get_container_element[x].find(class_='address').get_text().replace('\ue92f','')

        price = get_container_element[x].find(class_='price').get_text().replace('\ue92f','').replace(' ','')

        feature = get_container_element[x].find_all(class_='feature')

        bedroom = bedroom_regexr.findall(str(feature))[0] if bedroom_regexr.findall(str(feature)) else None

        bedrooms = bedrooms_regexr.findall(str(feature))[0] if bedrooms_regexr.findall(str(feature)) else None

        link = get_container_element[x].find(class_='shadow').find("a")['href']
        
        bathroom = bathroom_regexr.findall(str(feature))[0] if bathroom_regexr.findall(str(feature)) else None

        bathrooms = bathrooms_regexr.findall(str(feature))[0] if bathrooms_regexr.findall(str(feature)) else None


        floor = floor_regexr.findall(str(feature))[0] if floor_regexr.findall(str(feature)) else None
        floors = floors_regexr.findall(str(feature))[0] if floors_regexr.findall(str(feature)) else None

        land = land_regexr.findall(str(feature))[0] if land_regexr.findall(str(feature)) else None
        lands = lands_regexr.findall(str(feature))[0] if lands_regexr.findall(str(feature)) else None

        realestate_type = realestate_type.get_text().replace("$"," ").replace(" "," ").split(" ")[0] if realestate_type else None
        #============== End here ============================

        # conditional before push in list
        final_bedroom = bedrooms if bedrooms else bedroom
        final_bathroom = bathrooms if bathrooms else bathroom
        final_floor = floors if floors else floor
        final_land = lands if lands else land
        #============== End here ============================
        realestate_data_collection.append({'location': address, 'price': price, 'bedrooms': final_bedroom,
                     'bathrooms': final_bathroom, 'floors': final_floor, 'lands': final_land,
                     'realestate_category': realestate_type,'type': sell_or_rent, 'link': 'https://www.realestate.com.kh'+link
                    })

## Resuse the function

### Scrape the buy data set.

In [55]:
url = 'https://www.realestate.com.kh/buy'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

# https://www.realestate.com.kh/buy/?page=2

for page in range (50):
    extra_name_location_price_info('https://www.realestate.com.kh/buy/?page='+str(page+1), headers,'buy')
    if page == 50:
        print('done')


In [56]:
realestate_data_collection

[{'location': 'Veal Sbov, Chbar Ampov, Phnom Penh',
  'price': '$236,000',
  'bedrooms': '8',
  'bathrooms': '10',
  'floors': None,
  'lands': None,
  'realestate_category': 'Shophouse',
  'type': 'buy',
  'link': 'https://www.realestate.com.kh/buy/veal-sbov/8-bed-10-bath-shophouse-232724/'},
 {'location': 'Tonle Bassac, Chamkarmon, Phnom Penh',
  'price': '$840,403',
  'bedrooms': '3',
  'bathrooms': '3',
  'floors': None,
  'lands': None,
  'realestate_category': 'Condo',
  'type': 'buy',
  'link': 'https://www.realestate.com.kh/buy/tonle-bassac/3-bed-3-bath-condo-232673/'},
 {'location': 'Koh Dach, Chroy Changvar, Phnom Penh',
  'price': '$150/m²',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '792',
  'realestate_category': 'Land',
  'type': 'buy',
  'link': 'https://www.realestate.com.kh/buy/koh-dach/land-232168/'},
 {'location': 'Ream, Prey Nob, Sihanoukville',
  'price': '$1,150,000',
  'bedrooms': '3',
  'bathrooms': '5',
  'floors': '704',
  'lands': N

## Scrape the loan data

In [57]:
url = 'https://www.realestate.com.kh/rent'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

# https://www.realestate.com.kh/buy/?page=2

for page in range (50):
    extra_name_location_price_info('https://www.realestate.com.kh/rent/?page='+str(page+1), headers, 'rent')
    if page == 50:
        print('done')


## Save and load data

In [58]:
len(realestate_data_collection)


2000

In [59]:
def save_data(title,data):
    with open(title, "w",encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=3)

In [60]:
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [61]:
# saving data to json formate
realestate_kh = "./data/realestate_kh_50_pages_"+str(date.today())+".json"
save_data(realestate_kh,realestate_data_collection)
realestate_kh_list = load_data(realestate_kh)

## Save it CSV

In [62]:
df = pd.read_json(realestate_kh)
df.to_csv('./data/realestate_kh_50_pages_'+str(date.today())+'.csv')